In [2]:
"""首先创建一个DataFrame"""
import pandas as pd
df = pd.DataFrame({'a':[x//2 for x in range(5)],
                   'b':[x for x in range(5)],
                   'c':[x for x in range(5-1, -1, -1)]})
df

,a,b,c
0,0,0,4
1,0,1,3
2,1,2,2
3,1,3,1
4,2,4,0


In [5]:
"""Example1：a列所有的值+10"""
df1=df.copy()
df1['a']=df1['a'].apply(lambda x:x+10)
df1

,a,b,c
0,10,0,4
1,10,1,3
2,11,2,2
3,11,3,1
4,12,4,0


In [6]:
"""Example2：a列第1~3行的值+10"""
df2=df.copy()
df2.loc[1:3,'a']=df2.loc[1:3,'a'].apply(lambda x:x+10)
df2

,a,b,c
0,0,0,4
1,10,1,3
2,11,2,2
3,11,3,1
4,2,4,0


In [7]:
"""Example3：b列中小于c列、或大于k的值+10"""
df3=df.copy()
k=3
index=df3.query('b<c|b>@k').index.values
df3.loc[index,'b']=df3.loc[index,'b'].apply(lambda x: x+10)
df3

,a,b,c
0,0,10,4
1,0,11,3
2,1,2,2
3,1,3,1
4,2,14,0


In [8]:
"""Example4：新增d列，且d=a+c+10"""
df4=df.copy()
df4['d']=df4.apply(lambda col:col['a']+col['c']+10,axis=1)
df4

,a,b,c,d
0,0,0,4,14
1,0,1,3,13
2,1,2,2,13
3,1,3,1,12
4,2,4,0,12


In [9]:
"""Example5：a列的值改为b列-10，新增d列，且值为c列+10"""

df5 = df.copy()

# 同时修改多列的值时，是在操作DataFrame的一个区间，如果不新增列直接操作DataFrame，会因为无对应区间而产生bug

df5['d'] = None

# apply函数返回的是一个Series，不能将Series直接赋值给DataFrame，需要转化为List
# DataFrame接受的赋值格式为一个嵌套List：[[row1], [row2], ... [rowN]]
# 因此apply内的函数应该返回一个list,对应于[rowi]，apply返回的Series转化为List，对应于嵌套List的外层

df5[['a', 'd']] = df5.apply(lambda col:[col['b']-10, col['c']+10], axis=1).tolist()
df5


,a,b,c,d
0,-10,0,4,14
1,-9,1,3,13
2,-8,2,2,12
3,-7,3,1,11
4,-6,4,0,10


In [10]:
"""Example6：基于Example5的操作，但仅操作a列的值变化后小于-8的行, d列如果有空值，则填充为0"""
# 将Example6的条件写为函数, 注意返回值为列表
def func(a, b, c):
    if b-10 < -8:
        fa = b - 10
        fd = c + 10
    else:
        fa = a
        fd = 0
    return [fa, fd]

df6 = df.copy()
df6['d'] = None
df6[["a", "d"]]=df6.apply(lambda col:func(col['a'], col['b'], col['c']), axis=1).tolist()
df6


,a,b,c,d
0,-10,0,4,14
1,-9,1,3,13
2,1,2,2,0
3,1,3,1,0
4,2,4,0,0


In [24]:
# Pandas 详解二十六之 Apply-- 对行、列用函数处理

"""
俗话说，工欲善其事，必先利其器。在这里形容 apply 函数再合适不过了，apply 函数，可以说是 pandas 中自由度最高的函数。不过，是否能发挥其巨大威力，取决于我们的创造力。

在本博文中，首先通过简单例子来说明 apply 的用法，然后通过俩个高级例子来演示常用的用法。

1 简单例子来说明 apply 的用法
"""
df1=pd.DataFrame({'sex':list('FFMFMMF'),'smoker':list('YNYYNYY'),'age':[21,30,17,37,40,18,26],'weight':[120,100,132,140,94,89,123]})
df1

,sex,smoker,age,weight
0,F,Y,21,120
1,F,N,30,100
2,M,Y,17,132
3,F,Y,37,140
4,M,N,40,94
5,M,Y,18,89
6,F,Y,26,123


In [25]:
def bin_age(age):
    if age >=18:
        return 1
    else:
        return 0
df1['age'].apply(bin_age)


0    1
1    1
2    0
3    1
4    1
5    1
6    1
Name: age, dtype: int64

In [26]:
# df1['age'].apply(lambda x : 1 if x >18 else 0)

In [27]:
# 2 取出抽烟和不抽烟的体重前二
def top(df,col,n=5):
    return df.sort_values(by=col)[-n:]
df1.groupby('smoker').apply(topp,col='weight',n=2)

# df1 对象根据:’smoker’列，groupby 分组成抽烟和不抽烟二组，
# 然后每一组作为参数传进自定义的 top 函数，对于 top 函数的其余参数，
# 在 apply 函数作为关键字参数即可。对于自定义的 top 函数返回的 
# DataFrame 对象，将其所有返回的结果（在这是返回俩次）用 concat 连接，
# 形成最终结果。


sex smoker  age  weight
smoker                          
N      4   M      N   40      94
       1   F      N   30     100
Y      2   M      Y   17     132
       3   F      Y   37     140

In [28]:
"""
3 根据性别、年龄求每个人的含水量

据科学数据表明：

婴儿含水量 70%~80%

成年男性体内含水量 60%~65%

而成年女性体内含水量只是 50%~60%

（男人才是水做的～_~）
"""
def water(one_row):
    #婴儿
    if one_row['age']==1:
        return one_row['weight']*0.7
    #成年男女
    if one_row['sex']=='M':
        return one_row['weight']*0.6
    else:
        return one_row['weight']*0.5

df1.apply(water,axis=1)

0    60.0
1    50.0
2    79.2
3    70.0
4    56.4
5    53.4
6    61.5
dtype: float64

In [ ]:
"""
首先我们必须弄清楚，之前都是对列进行操作，而现在是求每一行，
也就是每个人的含水量，即对行进行操作，故应用 apply 函数时 axis=1。
而每一次应用自定义的 water 函数时，传进的一行是 Series 对象，
每次返回单值，最终组合成 Series 对象。

4 总结

首先，我们要弄清操作的元素是行还是列，若对每一行进行 apply，则 axis=1。其次，编写好 apply 应用的函数，注意传进的参数和返回值的类型是单值、Series 或 DataFrame。最后，groupby 和 apply 组合，运用得当将会产生巨大的威力！

"""

In [29]:
import pandas as pd
import numpy as np
 
table = pd.DataFrame({'cnt1':[1,2,np.nan,4,np.nan,6],
                      'cnt2':[5,10,np.nan,np.nan,6,10]})
 
#写成一句
table['sub'] = table.apply(lambda row: row['cnt1'] if (pd.isnull(row['cnt1']) | pd.isnull(row['cnt2'])) else (row['cnt1']-row['cnt2']),axis = 1 )

table

#写成函数
# def my_test(x,y):
#     if pd.isnull(x) | pd.isnull(y):
#         return x
#     else:
#         return x-y
# table['sub'] = table.apply(lambda row: my_test(row['cnt1'],row['cnt2']),axis = 1 )

,cnt1,cnt2,sub
0,1.0,5.0,-4.0
1,2.0,10.0,-8.0
2,NaN,NaN,NaN
3,4.0,NaN,4.0
4,NaN,6.0,NaN
5,6.0,10.0,-4.0
